<a href="https://colab.research.google.com/github/SBprjcts/DiffusionPolicyTraining/blob/main/PushT_TrainingNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Jan 21 04:32:12 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   69C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Cell 1: Install lerobot with PushT simulator support
!pip install 'lerobot[pusht]' -q

# After installation completes, you may need to restart the runtime
print("Installation complete! If you see dependency errors, go to Runtime → Restart runtime")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.0/821.0 MB 1.8 MB/s eta 0:00:00
 

In [4]:
!git clone https://github.com/huggingface/lerobot.git
%cd lerobot


Cloning into 'lerobot'...
remote: Enumerating objects: 39601, done.
remote: Counting objects: 100% (946/946), done.
remote: Compressing objects: 100% (389/389), done.
remote: Total 39601 (delta 791), reused 557 (delta 557), pack-reused 38655 (from 4)
Receiving objects: 100% (39601/39601), 233.99 MiB | 30.66 MiB/s, done.
Resolving deltas: 100% (25158/25158), done.
Filtering content: 100% (45/45), 69.03 MiB | 50.53 MiB/s, done.
/content/lerobot


In [5]:
!python examples/training/train_policy.py


Streaming output truncated to the last 5000 lines.
step: 0 loss: 1.121
step: 1 loss: 3.924
step: 2 loss: 1.118
step: 3 loss: 1.285
step: 4 loss: 1.025
step: 5 loss: 1.031
step: 6 loss: 1.097
step: 7 loss: 1.009
step: 8 loss: 1.092
step: 9 loss: 0.936
step: 10 loss: 0.975
step: 11 loss: 1.014
step: 12 loss: 0.990
step: 13 loss: 0.967
step: 14 loss: 0.945
step: 15 loss: 0.936
step: 16 loss: 0.994
step: 17 loss: 0.957
step: 18 loss: 0.958
step: 19 loss: 0.886
step: 20 loss: 0.903
step: 21 loss: 0.863
step: 22 loss: 0.899
step: 23 loss: 0.854
step: 24 loss: 0.768
step: 25 loss: 0.689
step: 26 loss: 0.649
step: 27 loss: 0.463
step: 28 loss: 0.542
step: 29 loss: 1.408
step: 30 loss: 0.765
step: 31 loss: 0.940
step: 32 loss: 0.749
step: 33 loss: 0.602
step: 34 loss: 0.535
step: 35 loss: 0.569
step: 36 loss: 0.370
step: 37 loss: 0.349
step: 38 loss: 0.416
step: 39 loss: 0.374
step: 40 loss: 0.274
step: 41 loss: 0.312
step: 42 loss: 0.326
step: 43 loss: 0.296
step: 44 loss: 0.242
step: 45 loss:

In [6]:
!lerobot-eval \
    --policy.path=outputs/train/example_pusht_diffusion \
    --env.type=pusht \
    --eval.batch_size=10 \
    --eval.n_episodes=10 \
    --policy.device=cuda


2026-01-21 05:26:31.963522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768973191.994466   14209 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768973192.005011   14209 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768973192.030511   14209 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768973192.030547   14209 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768973192.030554   14209 computation_placer.cc:177] computation placer alr

In [7]:
from IPython.display import Video
Video("outputs/eval/2026-01-21/05-26-41_pusht_diffusion/videos/pusht_0/eval_episode_6.mp4", embed=True)
